# Electricity mixes survey

This notebooks details the data preparation and plotting of chart showing electricity generation mixes of EU countries and the United Kingdom.


## Load prerequisities

This notebook uses the Tidyverse libraries and a couple of other packages. To install these libraries, run the command `install.packages(c("glue", "openssl", "tidyverse", "xml2", "yaml"))` in your R instance.


In [1]:
options(tidyverse.quiet = TRUE)
library(tidyverse)
library(xml2)

## Power generation data

For this survey, we use the electricity generation and emissions intensity data from the [Ember yearly electricity dataset](https://ember-climate.org/data-catalogue/yearly-electricity-data/). The Yearly Full Release Long Format” CSV file was downloaded into `/data/ember/yearly_full_release_long_format.csv`. The dataset is described as follows:

> The dataset contains yearly electricity generation, capacity, emissions, import and demand data for over 200 geographies. Data is collected from multi-country datasets (EIA, Eurostat, BP, UN) as well as national sources (e.g China data from the National Bureau of Statistics).

As we will need only annual electricity generation data for our analysis, and we will examine only the EU countries and the UK, we first need to filter the dataset accordingly.


In [15]:
EMBER_DATASET_PATH <- "../data/ember/yearly_full_release_long_format.csv"
SURVEY_TEXTS_PATH <- "../data/faktaoklimatu/electricity-mixes-country-stories.csv"
# Root URL for Wikimedia Commons public file storage.
COMMONS_URL_BASE <- "https://upload.wikimedia.org/wikipedia/commons"
# Path to the SVG template of the ternary plot.
TERNARY_TEMPLATE_SVG_PATH <- "../ternary/mix-template-path.svg"
# The time range to use for the electricity data.
YEAR_RANGE <- c(2000, 2021)
LAST_YEAR <- YEAR_RANGE[2]

survey_texts <- SURVEY_TEXTS_PATH |>
  read_csv(show_col_types = FALSE)

# Limit the following analyses from the Ember dataset
# to selected countries and year.
ember_common <- EMBER_DATASET_PATH |>
  read_csv(# Select only columns relevant for our analysis.
           col_select = c("Country code", "Area", "Year", "Category", "Variable", "Value", "Unit"),
           show_col_types = FALSE) |>
  rename(CountryCode = `Country code`) |>
  mutate(CountryCode = if_else(Area == "EU", "EU27", CountryCode)) |>
  filter(Year == LAST_YEAR,
         CountryCode %in% unique(survey_texts$CountryCode))

# Compute power generation figures from fossil, nuclear
# and renewable sources for each country.
power_generation <- ember_common |>
  filter(Category == "Electricity generation",
         Unit == "TWh",
         Variable %in% c("Fossil", "Nuclear", "Renewables")) |>
  select(CountryCode, Variable, Value) |>
  pivot_wider(names_from = Variable,
              values_from = Value) |>
  mutate(Total = Fossil + Nuclear + Renewables,
         FossilPct = 100 * Fossil / Total,
         NuclearPct = 100 * Nuclear / Total,
         RenewablesPct = 100 * Renewables / Total) |>
  # Join with net import figures.
  left_join(ember_common |>
              filter(Unit == "TWh", Variable == "Net Imports") |>
              select(CountryCode, NetImports = Value),
            by = "CountryCode")

# Pull out figures on CO₂ intensity of power generation.
power_emissions <- ember_common |>
  filter(Variable == "CO2 intensity") |>
  select(CountryCode, CO2Intensity = Value)

In [16]:
survey_joint <- left_join(survey_texts, power_generation, by = "CountryCode") |>
  left_join(power_emissions, by = "CountryCode") |>
  # Sort by country name according to Czech language collation rules.
  arrange(CountryName, .locale = "cs_CZ")

stopifnot(nrow(survey_joint) == nrow(power_generation))

head(survey_joint)

CountryCode,CountryName,FlagFilename,Text,Fossil,Renewables,Nuclear,Total,FossilPct,NuclearPct,RenewablesPct,NetImports,CO2Intensity
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AUS,Austrálie,Flag of Australia.svg,"Nejdůležitějším zdrojem elektřiny je uhlí, které dodává více než polovinu potřebné elektřiny. Podíl uhlí v mixu elektřiny postupně klesá, ovšem stále tvoří klíčový zdroj. Austrálie je zároveň [druhým největším exportérem uhlí na světě](https://www.statista.com/statistics/270952/global-hard-coal-exports-2009/). Dalším klíčovým zdrojem je zemní plyn, jehož je Austrálie [výrazným exportérem](https://www.ga.gov.au/digital-publication/aecr2021/gas). V posledních letech dochází k výraznému růstu produkce solární a větrné elektřiny, kdy Austrálie může využít své velmi vhodné přírodní podmínky. Vzhledem k těmto přírodním podmínkám se ovšem dá říct i to, že dosavadní rozvoj solární a větrné elektřiny nebyl dostatečný a nenaplňuje potenciál země. V současné době se plánuje propojení se Singapurem, kam by Austrálie do budoucna vyvážela elektřinu, ovšem [projekt se potýká s problémy](https://www.channelnewsasia.com/singapore/sun-cable-collapse-solar-energy-project-3200906).",175.08,71.98,0.00,247.06,70.86538,0.000000,29.13462,0.00,531.25
BEL,Belgie,Flag of Belgium.svg,"Belgie dodnes vyrábí zhruba polovinu své elektřiny v jaderných elektrárnách – v současnosti má [6 aktivních reaktorů](https://world-nuclear.org/information-library/country-profiles/countries-a-f/belgium.aspx). Plánovaný celkový phase-out do roku 2025 byl o 10 let odložen v důsledku současné energetické krize, přesto byl v září 2022 [uzavřen první reaktor od 80. let](https://www.world-nuclear-news.org/Articles/First-Belgian-power-reactor-shut-down). Belgie je zároveň prvním státem EU, který provedl roku 2016 celkový [uhelný phase-out](/infografiky/uhelny-phaseout-eu), ovšem nešlo vyloženě o klimaticky motivované rozhodnutí, důvodem bylo spíše stáří elektráren a evropské regulace týkající se znečištění ovzduší.",25.97,22.73,50.33,99.03,26.22438,50.822983,22.95264,-7.88,156.92
BRA,Brazílie,Flag of Brazil.svg,"Nejdůležitějším zdrojem elektřiny pro Brazíli jsou vodní elektrárny. V zemi funguje [druhá největší přehrada](https://www.weforum.org/agenda/2022/12/worlds-largest-hydroelectric-dams-renewable-energy/) na světě Itaipu, která se nachází na hranicích Brazílie a Paraguaye na řece Paraná. V brazilském mixu výroby elektřiny figuruje i [zemní plyn](https://fsr.eui.eu/natural-gas-in-brazils-energy-mix/), který z větší části pochází z domácí produkce. Důležitým zdrojem je též biomasa [pocházející převážně z vedlejších produktů](https://www.sciencedirect.com/science/article/abs/pii/S0959652622010873) ze zpracování cukrové třtiny. Brazílie momentálně staví svůj třetí [jaderný reaktor](https://www.world-nuclear.org/information-library/country-profiles/countries-a-f/brazil.aspx). V posledních letech dochází k výraznému nárůstu obnovitelných zdrojů pocházejících převážně z větrných elektráren. Vzhledem k přírodním podmínkám a emisně relativně nenáročnému mixu může být Brazílie do budoucna jedno z nejlepších míst na výrobu [zeleného vodíku](https://www.mckinsey.com/br/en/our-insights/all-insights/the-green-hidden-gem-brazils-opportunity-to-become-a-sustainability-powerhouse).",139.21,508.67,14.70,662.58,21.01029,2.218600,76.77111,23.10,158.59
BGR,Bulharsko,Flag of Bulgaria.svg,Bulharsko je stále do velké míry závislé na elektřině z uhlí (phase-out je plánován na rok [2038](https://www.euronews.com/green/2023/01/13/bulgaria-rolls-back-plans-to-phase-out-coal-amid-fears-over-energy-and-job-security)). Dalším významným zdrojem je [jádro](https://world-nuclear.org/information-library/country-profiles/countries-a-f/bulgaria.aspx) – země má dva reaktory a má plány postavit další čtyři. Výroba elektřiny ze solárních a větrných elektráren v poslední dekádě spíše stagnuje a k větší transformaci bulharské energetiky zatím nedochází.,20.50,10.31,16.49,4

In [17]:
make_commons_url <- function(filename) {
  filename_encoded <- gsub(" ", "_", filename)
  filename_hash <- openssl::md5(filename_encoded)
  file_url <- paste0(COMMONS_URL_BASE, "/",
                     substr(filename_hash, 1, 1), "/",
                     substr(filename_hash, 1, 2), "/",
                     filename_encoded)
  file_url
}

make_country_item <- function(...) {
  country <- list(...)
  flag_url <- make_commons_url(country$FlagFilename)
  country_item <- list(
    code = country$CountryCode,
    name = country$CountryName,
    `flag-url` = flag_url,
    fossil = round(country$FossilPct, 1),
    nuclear = round(country$NuclearPct, 1),
    renewables = round(country$RenewablesPct, 1),
    production = round(country$Total, 1),
    `net-imports` = round(country$NetImports, 1),
    intensity = signif(country$CO2Intensity, 3),
    text = country$Text
  )

  # Net imports may  undefined for EU-27.
  discard(country_item, is.na)
}

In [18]:
# Transform the country data into YAML and save to a file.
list(items = pmap(survey_joint, make_country_item)) |>
  yaml::as.yaml() |>
  writeLines("../outputs/electricity-mixes-survey.yml")

## Automatic triangle SVG generation

**Goal:** Given a template `ternary-template.svg`, generate `{CountryCode}.svg` for a given country.

**Computation steps:**

1. Take the dataset created above.
2. Compute _x_ and _y_ coordinates on the unit scale for each country and year.
3. Find out bounding box of the trinagle in the template (_x_ and _y_ position + width and height).
4. Transform the normalised (unit-scaled) _x_ and _y_ coordinates to the template's drawing coordinates.
5. Generate an SVG fragment for the path and the final bubble.
6. Adjust the labels and connectors.
7. Substitute the fragment into the SVG and save into `{CountryCode}.svg`.


In [6]:
power_generation_over_time <- EMBER_DATASET_PATH |>
  read_csv(# Select only columns relevant for our analysis.
           col_select = c("Country code", "Area", "Year", "Category", "Variable", "Value", "Unit"),
           show_col_types = FALSE) |>
  rename(CountryCode = `Country code`) |>
  mutate(CountryCode = if_else(Area == "EU", "EU27", CountryCode)) |>
  filter(YEAR_RANGE[1] <= Year, Year <= YEAR_RANGE[2],
         CountryCode %in% unique(survey_texts$CountryCode),
         Category == "Electricity generation",
         Unit == "TWh",
         Variable %in% c("Fossil", "Nuclear", "Renewables")) |>
  select(CountryCode, Year, Variable, Value) |>
  pivot_wider(names_from = Variable,
              values_from = Value) |>
  mutate(Total = Fossil + Nuclear + Renewables,
         FossilPct = 100 * Fossil / Total,
         NuclearPct = 100 * Nuclear / Total,
         RenewablesPct = 100 * Renewables / Total)

In [7]:
colour <- scales::colour_ramp(c("#b5b7ba", "#414042"))
year_to_colour <- function(year) {
  colour((year - YEAR_RANGE[1]) / (diff(YEAR_RANGE) - 1))
}

RADIUS_RANGE <- c(3, 8)
RADIUS_DOMAIN <- c(0, max(power_generation_over_time$Total))
radius_scale <- function(x) {
  RADIUS_RANGE[1] + diff(RADIUS_RANGE) * sqrt(x / RADIUS_DOMAIN[2])
}

power_generation_coords <- power_generation_over_time |>
  mutate(x = RenewablesPct + FossilPct / 2,
         y = 100 - FossilPct)

In [8]:
build_country_path <- function(country_df, bbox) {
  origin_years <- head(country_df$Year, -1)

  bbox_x <- as.numeric(xml_attr(bbox, "x"))
  bbox_y <- as.numeric(xml_attr(bbox, "y"))
  bbox_width <- as.numeric(xml_attr(bbox, "width"))
  bbox_height <- as.numeric(xml_attr(bbox, "height"))

  tibble(x1 = bbox_x + head(country_df$x, -1) / 100 * bbox_width,
         y1 = bbox_y + head(country_df$y, -1) / 100 * bbox_height,
         x2 = bbox_x + tail(country_df$x, -1) / 100 * bbox_width,
         y2 = bbox_y + tail(country_df$y, -1) / 100 * bbox_height,
         stroke = year_to_colour(origin_years)) |>
    pmap(function(x1, y1, x2, y2, stroke) {
      structure(list(), x1 = x1, y1 = y1, x2 = x2, y2 = y2,
                stroke = stroke, `stroke-linecap` = "round", `stroke-width` = "3")
    })
}

In [9]:
group_split(power_generation_coords, CountryCode) |>
  walk(function(country_df) {
    country_code <- first(country_df$CountryCode)
    num_lines <- nrow(country_df) - 1

    # TODO: Open only once, then clone as needed, if possible.
    template_svg <- read_xml(TERNARY_TEMPLATE_SVG_PATH)

    bbox <- xml_find_first(template_svg, "//d1:g[@id='bbox']/d1:rect")

    g_path <- xml_find_first(template_svg, "//d1:g[@id='path']")

    # Add path to the ternary plot.
    line_nodes <- build_country_path(country_df, bbox)
    names(line_nodes) <- rep_along(line_nodes, "line")
    new_path <- as_xml_document(
      list(g = structure(line_nodes, id = "path"))
    )

    xml_replace(g_path, new_path, free = TRUE)
    xml_remove(bbox)

    # Add bubble for the final year.
    bubble_radius <- country_df$Total |> last() |> radius_scale()
    last_bubble <- as_xml_document(
      list(ellipse = structure(list(),
                               cx = attr(line_nodes[[num_lines]], "x2") |> as.character(),
                               cy = attr(line_nodes[[num_lines]], "y2") |> as.character(),
                               rx = as.character(bubble_radius),
                               ry = as.character(bubble_radius),
                               fill = "#ffffff",
                               stroke = "#414042"))
    )
    xml_add_child(template_svg, last_bubble, copy = FALSE)

    # Adjust labels and connectors.
    first_label <- xml_find_first(template_svg, "//d1:g[@id='first-year-label']/d1:text")
    last_label <- xml_find_first(template_svg, "//d1:g[@id='last-year-label']/d1:text")
    xml_text(first_label) <- as.character(YEAR_RANGE[1])
    xml_text(last_label) <- as.character(YEAR_RANGE[2])
    first_label_coords <- list(x = (attr(line_nodes[[1]], "x1") - 10) |> round(2),
                               y = (attr(line_nodes[[1]], "y1") + 4) |> round(2))
    last_label_coords <- list(x = (attr(line_nodes[[num_lines]], "x2") + bubble_radius + 10) |> round(2),
                              y = (attr(line_nodes[[num_lines]], "y2") + 4) |> round(2))
    xml_attr(first_label, "transform") <- glue::glue("translate({first_label_coords$x}, {first_label_coords$y})")
    xml_attr(last_label, "transform") <- glue::glue("translate({last_label_coords$x}, {last_label_coords$y})")

    first_connector <- xml_find_first(template_svg, "//d1:line[@id='first-year-connector']")
    xml_attr(first_connector, "x1") <- as.character(attr(line_nodes[[1]], "x1") - 8)
    xml_attr(first_connector, "y1") <- as.character(attr(line_nodes[[1]], "y1"))
    xml_attr(first_connector, "x2") <- as.character(attr(line_nodes[[1]], "x1"))
    xml_attr(first_connector, "y2") <- as.character(attr(line_nodes[[1]], "y1"))

    last_connector <- xml_find_first(template_svg, "//d1:line[@id='last-year-connector']")
    xml_attr(last_connector, "x1") <- as.character(attr(line_nodes[[num_lines]], "x2") + bubble_radius + 8)
    xml_attr(last_connector, "y1") <- as.character(attr(line_nodes[[num_lines]], "y2"))
    xml_attr(last_connector, "x2") <- as.character(attr(line_nodes[[num_lines]], "x2"))
    xml_attr(last_connector, "y2") <- as.character(attr(line_nodes[[num_lines]], "y2"))

    # Save the SVG.
    write_xml(template_svg, glue::glue("../ternary/svg/{country_code}.svg"))
  })